# Day 14: Reindeer Olympics

[*Advent of Code 2015 day 14*](https://adventofcode.com/2015/day/14) and [*solution megathread*](https://www.reddit.com/3wqtx2)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/14/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F14%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

...

In [5]:
testdata = """Comet can fly 14 km/s for 10 seconds, but then must rest for 127 seconds.
Dancer can fly 16 km/s for 11 seconds, but then must rest for 162 seconds."""

inputdata = downloaded['input']

1:80: E501 line too long (87 > 79 characters)


In [6]:
inputdata[:100]

'Rudolph can fly 22 km/s for 8 seconds, but then must rest for 165 seconds.\nCupid can fly 8 km/s for '

In [7]:
import re


def parse_data(data: str):
    output = []
    pattern = re.compile(
        r'^(?P<deer>\w+) can fly (?P<speed>\d+) km/s for (?P<endurance>\d+) ' +
        r'seconds, but then must rest for (?P<holdoff>\d+) seconds\.$')
    for line in data.splitlines():
        matches = pattern.match(line)
        output.append(matches.groupdict())
    return output


def performance_to_state(perf, debug=False):
    for intfield in ['speed', 'endurance', 'holdoff']:
        perf[intfield] = int(perf[intfield])
    if debug:
        print(perf)
    return perf | {
        'location': 0,
        'holdoff_r': 0,
        'endurance_r': perf['endurance']}


def update_state(state, delta=1, debug=False):
    if state['holdoff_r'] > 0:
        if state['holdoff_r'] <= delta:
            delta_r = delta - state['holdoff_r']
            if debug:
                print(f'case 1: {state["deer"]} has rested ' +
                      f'{state["holdoff_r"]} seconds, now racing ' +
                      f'another {delta_r} seconds')
            state['holdoff_r'] = 0
            state['endurance_r'] = state['endurance']
            if delta_r > 0:
                update_state(state, delta_r, debug)
            return
        state['holdoff_r'] -= delta
        if debug:
            print(f'case 2: {state["deer"]} is resting another {delta} ' +
                  f'seconds, {state["holdoff_r"]} seconds rest remaining')
        return
    elif state['endurance_r'] > delta:
        state['location'] += delta * state['speed']
        state['endurance_r'] -= delta
        if debug:
            print(f'case 3: {state["deer"]} raced {delta} seconds, now in ' +
                  f'location {state["location"]} with a remaining endurance ' +
                  f'of {state["endurance_r"]}')
        return
    else:
        delta_t = state['endurance_r']
        delta_n = delta - delta_t
        state['endurance_r'] = 0
        state['location'] += delta_t * state['speed']
        state['holdoff_r'] = state['holdoff']
        if debug:
            print(f'case 4: {state["deer"]} raced {delta_t} seconds, now in ' +
                  f'location {state["location"]}, will rest for ' +
                  f'{state["holdoff_r"]} with a remaining {delta_n} ' +
                  'seconds to race')
        update_state(state, delta_n, debug)
        return


def race_deer(states, duration=1000, debug=False):
    for state in states:
        update_state(state, delta=duration, debug=debug)
    return [{"deer": state["deer"],
            "location": state["location"]} for state in states]


def my_part1_solution(data, duration=1000, debug=False):
    states = [performance_to_state(p, debug) for p in parse_data(data)]
    return race_deer(states, duration=duration, debug=debug)

In [8]:
my_part1_solution(testdata, duration=1000, debug=True)

{'deer': 'Comet', 'speed': 14, 'endurance': 10, 'holdoff': 127}
{'deer': 'Dancer', 'speed': 16, 'endurance': 11, 'holdoff': 162}
case 4: Comet raced 10 seconds, now in location 140, will rest for 127 with a remaining 990 seconds to race
case 1: Comet has rested 127 seconds, now racing another 863 seconds
case 4: Comet raced 10 seconds, now in location 280, will rest for 127 with a remaining 853 seconds to race
case 1: Comet has rested 127 seconds, now racing another 726 seconds
case 4: Comet raced 10 seconds, now in location 420, will rest for 127 with a remaining 716 seconds to race
case 1: Comet has rested 127 seconds, now racing another 589 seconds
case 4: Comet raced 10 seconds, now in location 560, will rest for 127 with a remaining 579 seconds to race
case 1: Comet has rested 127 seconds, now racing another 452 seconds
case 4: Comet raced 10 seconds, now in location 700, will rest for 127 with a remaining 442 seconds to race
case 1: Comet has rested 127 seconds, now racing anothe

[{'deer': 'Comet', 'location': 1120}, {'deer': 'Dancer', 'location': 1056}]

In [9]:
results = my_part1_solution(inputdata, duration=2503, debug=False)
print(results)
print(max(result["location"] for result in results))

[{'deer': 'Rudolph', 'location': 2640}, {'deer': 'Cupid', 'location': 2696}, {'deer': 'Prancer', 'location': 2484}, {'deer': 'Donner', 'location': 2550}, {'deer': 'Dasher', 'location': 2508}, {'deer': 'Comet', 'location': 2520}, {'deer': 'Blitzen', 'location': 2592}, {'deer': 'Vixen', 'location': 2560}, {'deer': 'Dancer', 'location': 2527}]
2696


In [10]:
HTML(downloaded['part1_footer'])

## Part Two

In [11]:
HTML(downloaded['part2'])

In [12]:
def score_states(states, debug=False):
    lead_location = max(state["location"] for state in states)
    for state in states:
        if state["location"] == lead_location:
            state["score"] += 1
            if debug:
                print(f'{state["deer"]} scored this second for ' +
                      f'a score of {state["score"]}')


def my_part2_solution(data, duration=1000, debug=False):
    states = [performance_to_state(p, debug) for p in parse_data(data)]
    for state in states:
        state["score"] = 0
    for _ in range(duration):
        _ = race_deer(states, duration=1, debug=debug)
        score_states(states, debug)
    if debug:
        print(states)
    return max(state["score"] for state in states)

In [13]:
my_part2_solution(testdata, duration=1000, debug=False)

689

In [14]:
my_part2_solution(inputdata, duration=2503, debug=False)

1084

In [15]:
HTML(downloaded['part2_footer'])